<a href="https://colab.research.google.com/github/michelleaeh/ADissertation/blob/master/2_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project:** MSc in Robotics and Intelligent Systems Dissertation

**Project name:**

**Author:** Michelle Alejandra Espinosa Hernandez

**Student registration number:** 1900964

**Student PRID:** ESPIN62803

**Date created:** 

**Purpose:** Obtain classification accuracy among different types of sensors and different data processing steps

# **Data description**

**Myo armband dataset from https://data.mendeley.com/datasets/wgswcr8z24/2**


The dataset consits of .csv files collected from two Myo armbands. The format of the files are [word_name]_[id]. The ‘word_name’ is the English translation of the American Sign Language word used and the ‘id’ is a unique identifier. The .zip for each of the above links has sub-folders for each User.

Each file has 50 columns. They represent a sub-sampled data collection from two Myo devices worn on left and right hands of the signer. The first column is the ‘Counter’ that goes from 1 to 50.

The following columns are of the format: [Sensor][pod/direction][left/right]. For instance the EMG reading for the first EMG pod (out of 8) on the left hand would be called EMG0R and the accelerometer reading for the Z axis on the left hand would be called: AXL

If you use this dataset please cite the following papers:

@inproceedings{paudyal2016sceptre,
title={Sceptre: a pervasive, non-invasive, and programmable gesture recognition technology},
author={Paudyal, Prajwal and Banerjee, Ayan and Gupta, Sandeep KS},
booktitle={Proceedings of the 21st International Conference on Intelligent User Interfaces},
pages={282--293},
year={2016},
organization={ACM}
}

@inproceedings{paudyal2017dyfav,
title={Dyfav: Dynamic feature selection and voting for real-time recognition of fingerspelled alphabet using wearables},
author={Paudyal, Prajwal and Lee, Junghyo and Banerjee, Ayan and Gupta, Sandeep KS},
booktitle={Proceedings of the 22nd International Conference on Intelligent User Interfaces},
pages={457--467},
year={2017},
organization={ACM}
}

**Frequency:**

50Hz sampling rate

**Words:**

*36 total words*

allmorning, bird, blue, cantsleep, cat, colrunnynose, continuouslyforanhour, cost, day, dollar, everymorning, everynight, gold, goodnight, happy, headache, home, horse, hot, hurt, itching, large, mom, monthly, notfeelgood, orange, pizza, please, shirt, soreness, swelling, takeliquidmedicine, thatsterrible, tired, upsetstomach, wash


**Filenames:**

*849 total files*

(word)_(user#)(try#)


**Columns of files:**

Counter  (1 -> 50)

EMG0L -> EMG7L  (EMG sensor readings)

AXL, AYL, AZL  (accelerometer readings)

GXL, GYL, GZL  (gyroscope readings)

ORL, OPL, OYL  (magnetometer readings?)

EMG0R -> EMG7R  (EMG sensor readings)

AXR, AYR, AZR  (accelerometer readings)

GXR, GYR, GZR  (gyroscope readings)

ORR, OPR, OYR  (magnetometer readings?)

features=['EMG0L', 'EMG1L', 'EMG2L', 'EMG3L', 'EMG4L', 'EMG5L', 'EMG6L', 'EMG7L', 'AXL', 'AYL', 'AZL', 'GXL', 'GYL', 'GZL', 'ORL', 'OPL', 'OYL', 'EMG0R', 'EMG1R', 'EMG2R', 'EMG3R', 'EMG4R', 'EMG5R', 'EMG6R', 'EMG7R', 'AXR', 'AYR', 'AZR', 'GXR', 'GYR', 'GZR', 'ORR', 'OPR', 'OYR']


**Size of files:**

All files are 50 rows x 35 columns except continuouslyforanhour_22.csv, headache_52.csv, home_61.csv, and mom_82.csv which are 101 rows x 35 columns

**Steps:**

1. Combine files
2. Normalize or standardize matrix
3. Apply Butterworth
4. Apply PCA
5. Input to SVM



# **1. Preparation of data**

**1.1. Start up and initialization of variables**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pandas as pd
import scipy as sp
import warnings
warnings.filterwarnings("ignore")
import timeit
start = timeit.default_timer()

from collections import Counter
from google.colab import files
from mpl_toolkits import mplot3d
from scipy import signal
from scipy.io import loadmat
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from zipfile import ZipFile

# Extract all files from zip
with ZipFile("/content/2MyoASL.zip", 'r') as zip:
  zip.extractall()

products=[3, 5, 7, 11, 15, 21, 33, 35, 55, 77, 105, 165, 231, 385, 1155] # Generate matrices for all combinations of sensors (E=3, A=5, G=7, O=11)
steps=[13, 39, 65, 91, 143, 273, 429, 455, 715, 1001, 3003, 5005] # Generate matrices for all combinations of steps (Normalization=3, Standardization=5, Butterworth=7, PCA=11, SVM=13)
comb=['E', 'A', 'G', 'O', 'EA', 'EG', 'EO', 'AG', 'AO', 'GO', 'EAG', 'EAO', 'EGO', 'AGO', 'EAGO'] # Generate matrices for all combinations of sensors (E=emg, A=acc, G=gyro, O=ori)
nsteps=['V', 'NV', 'SV', 'BV', 'PV', 'NBV', 'NPV', 'SBV', 'SPV', 'BPV', 'NBPV', 'SBPV'] # Generate matrices for all combinations of steps (N=Normalization, S=Standardization, B=Butterworth, P=PCA, V=SVM)
emg=['EMG0L', 'EMG1L', 'EMG2L', 'EMG3L', 'EMG4L', 'EMG5L', 'EMG6L', 'EMG7L', 
     'EMG0R', 'EMG1R', 'EMG2R', 'EMG3R', 'EMG4R', 'EMG5R', 'EMG6R', 'EMG7R']
acc=['AXL', 'AYL', 'AZL', 'AXR', 'AYR', 'AZR']
gyro=['GXL', 'GYL', 'GZL', 'GXR', 'GYR', 'GZR']
ori=['ORL', 'OPL', 'OYL', 'ORR', 'OPR', 'OYR']

# Initialization of counters
words=['allmorning', 'bird', 'blue', 'cantsleep', 'cat', 'coldrunnynose', 'continuouslyforanhour', 'cost', 'day', 
       'dollar', 'everymorning', 'everynight', 'gold', 'goodnight', 'happy', 'headache', 'home', 'horse', 'hot', 
       'hurt', 'itching', 'large', 'mom', 'monthly', 'notfeelgood', 'orange', 'pizza', 'please', 'shirt', 
       'soreness', 'swelling', 'takeliquidmedicine', 'thatsterrible', 'tired', 'upsetstomach', 'wash']
colnames=emg[:8]+acc[:3]+gyro[:3]+ori[:3]+emg[8:]+acc[3:]+gyro[3:]+ori[3:]
lengths=np.zeros(849, dtype=int)
reps=np.zeros(36,dtype=int)
repsum=np.zeros(37,dtype=int)
headers=np.empty(1701, dtype=object)
fresults=np.zeros((len(steps),len(products)))
params=np.zeros((len(steps),len(products)))
features=np.zeros(15)
target=np.zeros(15)
matrix=np.zeros(1)
norm=[]
stand=[]

fn=np.arange(1701)
wordnum=-1
counter=-1
rownum=-1
start=0
num=0
n=0

**1.2. Combine all files**

In [ ]:
for w in words:
  repcount=0
  wordnum+=1
  for i in range (10, 120):
    path='/content/2MyoASL/' + w + '_' + str(i) + '.csv'
    if os.path.exists(path)==True:
      counter+=1
      repcount+=1
      trial=pd.read_csv(path)
      trial.reset_index(drop=True)
      
      # Assign word number to one row
      row=np.zeros(1)
      for t in range(35):
        if t==0:
          row[0]=wordnum
        else:
          sensor=trial.iloc[0:50,t].values
          sensor.reshape([1,50])
          row=np.concatenate((row, sensor))
      prev=row
      
      # Combine all trials
      if counter==0:
        matrix=prev
      else:
        matrix=np.concatenate([matrix,prev])

  reps[wordnum]=repcount
  if wordnum>0:
    repsum[wordnum]=reps[wordnum-1]+repsum[wordnum-1]
    repsum[36]=849

# Create header name array
headers[0]='Word'
for c in colnames:
  for t in range(50):
    num+=1
    headers[num]=c

# Give format to final matrix 
matrix=matrix.reshape([849,1701])
matrix=pd.DataFrame(matrix, columns=headers)
print(matrix)
print('Reps of each word:',reps)
print('Cummulative reps:',repsum)

     Word  EMG0L  EMG0L  EMG0L  EMG0L  ...    OYR    OYR    OYR    OYR    OYR
0     0.0    0.0    0.0   -3.0   -1.0  ...   83.0   83.0   83.0   83.0   83.0
1     0.0    0.0    0.0    4.0   -2.0  ...   83.0   83.0   83.0   83.0   83.0
2     0.0    0.0    0.0   -8.0   -1.0  ...   84.0   84.0   84.0   84.0   84.0
3     0.0    0.0    0.0   -1.0  -19.0  ...   83.0   83.0   83.0   83.0   83.0
4     0.0    0.0    0.0    1.0  -16.0  ...   93.0   91.0   91.0   91.0   91.0
..    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
844  35.0    0.0    0.0   -2.0    0.0  ...  106.0  104.0  100.0   98.0   95.0
845  35.0    0.0    0.0    2.0   -8.0  ...  176.0  176.0  177.0  177.0  177.0
846  35.0    0.0    0.0   -8.0   26.0  ...  177.0  176.0  175.0  175.0  176.0
847  35.0    0.0    0.0   -1.0   21.0  ...  178.0  178.0  179.0  179.0  179.0
848  35.0    0.0    0.0    8.0  -19.0  ...  179.0  179.0  179.0  179.0    0.0

[849 rows x 1701 columns]
Reps of each word: [19 24 32 20 24 20

**1.3. Calculate mean and standard deviation of each sensor and each file**

In [ ]:
# Average and standard deviation of each sensor in each file
for s in colnames:
  avg=matrix[s].mean(axis=1)
  sd=matrix[s].std(axis=1)
  sensor=pd.concat([avg.rename(s+': Mean_'),sd.rename('St. dev.')], axis=1)
  if s=='EMG0L':
    asd=sensor
  else:
    asd=pd.concat([asd, sensor], axis=1)
print(asd)

# Average and standard deviation of each file
avg=matrix.mean(axis=1)
sd=matrix.std(axis=1)
pd.concat([avg.rename('Mean'),sd.rename('St. dev.')], axis=1)

     EMG0L: Mean_   St. dev.  EMG1L: Mean_  ...   St. dev.  OYR: Mean_   St. dev.
0           -2.32   8.664825         -0.80  ...  28.427609       86.70  10.529356
1           -1.80   9.544739         -4.08  ...  27.821010       85.68   6.579002
2           -3.16  13.085839         -2.32  ...  30.345736       91.88  17.358924
3           -0.82  10.123099         -3.16  ...  29.645002       88.46  12.969682
4           -0.50   6.516071          2.76  ...  13.237239      116.58  18.099600
..            ...        ...           ...  ...        ...         ...        ...
844          0.56   7.754288          0.22  ...  17.236281      104.76   7.528341
845         -1.74   6.520955         -1.20  ...  21.772891       69.66  85.251036
846          1.02   7.731436          0.46  ...  22.707735       56.76  79.408415
847          0.06   5.582078          2.36  ...  23.650422       71.62  84.238701
848         -2.22   7.434860         -1.62  ...  25.101622       58.52  79.668763

[849 rows x 68 

,Mean,St. dev.
0,15.256437,46.545832
1,14.220110,47.316822
2,15.441784,45.614456
3,13.651092,47.038916
4,15.063294,49.105612
...,...,...
844,13.167964,41.642351
845,13.815314,42.880121
846,12.970618,42.725653
847,13.852317,43.758631


# **2. Variable modification**

**2.1. Establish equal number of repetitions per word**

In [ ]:
# Ensure all words have same number of repetitions
numreps=2     # Number of repetitions per word
num_trials=10  # Number of runs of cross validation

for i in range(len(reps)-1,-1,-1):
  tl=sd.iloc[repsum[i]:repsum[i+1]]
  u=repsum[i+1]-1
  if reps[i]<numreps:
    for r in range(len(matrix)-1,-1,-1):
      if int(matrix.iloc[r]['Word'])==i:
        matrix=matrix.drop(r)
  elif reps[i]>numreps:
    while reps[i]>numreps:
      if tl[u]==tl.max():
        tl[u]=0
        matrix=matrix.drop(u)
        reps[i]=reps[i]-1
        u=repsum[i+1]-1
      else:
        u-=1

print(matrix)
print('Repetitions per word:',reps)
exec("matrix.to_csv(path_or_buf='/content/matrix_'+str(numreps)+'.csv')")

     Word  EMG0L  EMG0L  EMG0L  EMG0L  ...   OYR   OYR   OYR   OYR   OYR
13    0.0    0.0    0.0    3.0   10.0  ...  74.0  74.0  74.0  74.0  74.0
14    0.0    0.0    0.0   -8.0   -1.0  ...  75.0  74.0  75.0  75.0  75.0
31    1.0    0.0    0.0   10.0   13.0  ...  92.0  91.0  89.0  86.0  82.0
36    1.0    0.0    0.0   -2.0   -2.0  ...  31.0  31.0  31.0  31.0  31.0
47    2.0    0.0    0.0   -3.0   -1.0  ...  39.0  41.0  44.0  44.0  44.0
..    ...    ...    ...    ...    ...  ...   ...   ...   ...   ...   ...
786  33.0    0.0    0.0    5.0  -12.0  ...  86.0  86.0  85.0  85.0  85.0
808  34.0    0.0    0.0    0.0    2.0  ...  81.0  82.0  82.0  82.0  82.0
810  34.0    0.0    0.0   -2.0   -1.0  ...  84.0  84.0  84.0  84.0  84.0
835  35.0    0.0    0.0   23.0    1.0  ...  68.0  66.0  65.0  63.0  63.0
836  35.0    0.0    0.0   -4.0   -8.0  ...  62.0  62.0  62.0  62.0  62.0

[72 rows x 1701 columns]
Repetitions per word: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


**2.2. Create sensor combinatory matrices: unaltered, normalized, and standardized**

In [ ]:
# Eliminate unnecessary columns to create combinatory matrices
for v in products:
  m=matrix.copy()
  if v%3!=0:
    m=m.drop(emg,1)
  if v%5!=0:
    m=m.drop(acc,1)
  if v%7!=0:
    m=m.drop(gyro,1)
  if v%11!=0:
    m=m.drop(ori,1)
  
  # Separate features from target values
  x = m.iloc[:, m.columns!='Word']   # Features

  # Create column of words instead of number
  wordcol=np.empty(len(m), dtype=object)
  z=0
  wcol=[int(i) for i in m['Word'].values]
  for f in wcol:
    wordcol[z]=words[f]
    z+=1
  wordcol=np.asmatrix(wordcol)
  regular=np.concatenate((np.transpose(wordcol),np.copy(m.iloc[:,1:])),axis=1)
  products[n]=pd.DataFrame(regular,columns=m.columns).dropna(axis=1)
 
  ## Normalize features so that each column is between 0 and 1
  norm_matrix=(m-m.min())/(m.max()-m.min())
  norm_matrix=np.concatenate((np.transpose(wordcol),norm_matrix.iloc[:,1:]),axis=1)
  norm_matrix=pd.DataFrame(norm_matrix,columns=m.columns).dropna(axis=1)
  norm.append(norm_matrix)
  
  ## Standardize features with mean=0 and deviation=1
  standardized_matrix=StandardScaler().fit_transform(x)
  standardized_matrix=np.concatenate((np.transpose(wordcol),standardized_matrix),axis=1)
  standardized_matrix=pd.DataFrame(standardized_matrix,columns=m.columns).dropna(axis=1)
  stand.append(standardized_matrix)
  n+=1

# **3. Definition of functions for steps**

**3.1. Split data into train and test sets**

In [ ]:
def datasplit(inmatrix_p):
    x = inmatrix_p.iloc[:, inmatrix_p.columns!='Word']   # Features
    y = inmatrix_p.loc[:,'Word']     # Target
    x_train_p, x_test_p, y_train_p, y_test_p = train_test_split(x, y, test_size=0.3)
    return x_train_p, x_test_p, y_train_p, y_test_p

**3.2. Butterworth**

In [ ]:
def butterworth(inmatrix_b):
  high = 1/(50/2)
  low = 23/(50/2)

  b, a = sp.signal.butter(7, [high,low], btype='bandpass')

  for r in emg:
    if r in inmatrix_b:
      # process EMG signal: filter EMG
      emg_filtered = sp.signal.lfilter(b, a, inmatrix_b[[r]])
      inmatrix_b[[r]]=emg_filtered
  return inmatrix_b

**3.3. PCA**

In [ ]:
def pca(x_train_c, x_test_c, y_train_c, y_test_c):
  pca = PCA(n_components=min(len(x_train_c), len(y_train_c)))
  pca.fit(x_train_c)
  x_t_train_pca = pca.transform(x_train_c)
  x_t_test_pca = pca.transform(x_test_c)

  # Plot
  #print("Normalized matrix")
  #print(pca.explained_variance_ratio_)
  #print(pca.singular_values_)
  #plt.figure()
  #plt.bar(fn[:100], pca.explained_variance_ratio_)
  #plt.show()
  #plt.bar(fn[:100], pca.singular_values_)
  #plt.show()
  return x_train_c, x_test_c, y_train_c, y_test_c, x_t_train_pca, x_t_test_pca

**3.4. SVM with Grid Search**

In [ ]:
def svm(x_train_s, x_test_s, y_train_s, y_test_s, x_t_train_s, x_t_test_s,combo):
    parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[0.1, 1, 10, 100, 1000]}
    svc = SVC(max_iter=1000)
    #if int(numreps*0.3)<2:
    #  nsplit=2
    #else:
    #  nsplit=int(numreps*0.3)
    nested_scores=np.zeros(num_trials)
    for t in range(num_trials):
      inner_cv=KFold(n_splits=10,shuffle=True,random_state=i)
      outer_cv=KFold(n_splits=10,shuffle=True,random_state=i)
      #clf=GridSearchCV(svc,parameters,scoring='accuracy',n_jobs=1,cv=min(nsplit,10))
      clf=GridSearchCV(svc,parameters,scoring='accuracy',n_jobs=1,cv=outer_cv)
      clf.fit(x_t_train_s, y_train_s)
      nested_scores=cross_val_score(clf,x_t_test_s,y_test_s,cv=outer_cv)
      nested_scores[i]=nested_scores.mean()
    #print('score', clf.score(x_t_test_s, y_test_s))
    y_pred=clf.predict(x_t_test_s)
    #print('pred label', y_pred)
    #print('length',len(clf.predict(x_t_test_s)))

    # Confusion matrix
    #plot_confusion_matrix(clf, x_t_test_s, y_test_s,cmap=plt.cm.Blues)
    #plt.title(combo)
    #plt.figure(figsize=(50,50))
    #plt.show()
    bestpar=clf.best_params_
    accuracy=nested_scores.mean()
    svmresult=classification_report(y_test_s, y_pred)
    return svmresult,accuracy, bestpar

# **4. Main code**

In [10]:
for st in steps:
  rownum+=1
  columnnum=-1
  for pr in products:
    columnnum+=1
    combo=nsteps[rownum]+' - '+comb[columnnum]
    words=set(pr['Word']) # Obtain the target names for the SVM
    # First step: set input matrix as regular, normalized or standardized
    input_matrix=pr
    if st%3==0: # Normalization
      input_matrix=norm[columnnum]
    if st%5==0: # Standardization
      input_matrix=stand[columnnum]
    # Second step: apply Butterworth
    two_matrix=input_matrix
    if st%7==0: # Butterworth
      two_matrix=butterworth(input_matrix)
    # Third step: split data for later steps
    x_train, x_test, y_train, y_test=datasplit(two_matrix)
    # Fourth step: apply PCA
    x_t_train=x_train
    x_t_test=x_test
    if st%11==0: # PCA
      x_train, x_test, y_train, y_test, x_t_train, x_t_test=pca(x_train, x_test, y_train, y_test)
    # Fifth step: apply SVM
    if st%13==0: # SVM
      svmresults,accuracy,par=svm(x_train, x_test, y_train, y_test, x_t_train, x_t_test,combo)
    print('The best parameters for',combo,'are:',par)
    fresults[rownum,columnnum]=accuracy*100
fresults=pd.DataFrame(fresults,index=nsteps,columns=comb)
exec("fresults.to_csv(path_or_buf='/content/results_'+str(numreps)+'.csv')")

stop = timeit.default_timer()
execution_time = stop - start

The best parameters for V - E are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - A are: {'C': 100, 'kernel': 'sigmoid'}
The best parameters for V - G are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - O are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - EA are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - EG are: {'C': 10, 'kernel': 'sigmoid'}
The best parameters for V - EO are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - AG are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - AO are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - GO are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - EAG are: {'C': 10, 'kernel': 'sigmoid'}
The best parameters for V - EAO are: {'C': 0.1, 'kernel': 'linear'}
The best parameters for V - EGO are: {'C': 100, 'kernel': 'sigmoid'}
The best parameters for V - AGO are: {'C': 100, 'kernel': 'sigmoid'}
The best parameters for V - EAGO are: {'C': 100, 'kernel': 

# **5. Results**

In [11]:
print(fresults)
print('The maximum accuracy for',numreps,'repetitions is',fresults.max().max())
print("The program executed in "+str(execution_time))

             E          A          G  ...        EGO        AGO       EAGO
V     5.500000  37.333333   9.166667  ...  11.000000  11.000000  25.666667
NV    0.000000   0.000000   9.166667  ...  11.000000   5.500000  15.000000
SV    0.000000   5.833333   0.000000  ...   0.000000  20.166667   4.000000
BV    0.000000  20.166667  14.666667  ...  11.000000   5.500000   9.166667
PV    0.000000  16.833333   0.000000  ...  44.666667  27.833333  16.500000
NBV   0.000000  11.000000   0.000000  ...   5.500000  16.833333   0.000000
NPV   0.000000   0.000000   0.000000  ...  16.833333  26.000000  25.666667
SBV   5.500000  27.500000   0.000000  ...  11.000000  16.833333   3.666667
SPV   0.000000  11.000000  20.166667  ...   0.000000  16.500000  27.833333
BPV   0.333333   0.000000  16.833333  ...  16.500000  14.666667  22.666667
NBPV  0.000000  31.166667   0.000000  ...   5.500000  16.833333  27.833333
SBPV  0.000000   5.833333   0.333333  ...  20.500000  16.500000   5.833333

[12 rows x 15 columns]
T

# **To-do list**

-check what happens when combining diff data sources

-graph components

-find correlations

-interpret pca results

-try pca per individual/word

-read dataset papers

correlation component with word

-try straight to svm

-resend email

-check what is the data

-merge timeseries word index

-see how stable each feature are

-standard deviation timeseries

-try without filter

-multi-class classifier SVM

**
-try with 2 words, only emg
-average per column
-test each step
-test svm with 2 matrices
see if svm has access to test set 
maybe shifted per set 



# **Additional papers**

https://www.researchgate.net/publication/303707429_Combining_Smartphone_and_Smartwatch_Sensor_Data_in_Activity_Recognition_Approaches_an_Experimental_Evaluation

# **References**

Different parameters of SVC https://medium.com/all-things-ai/in-depth-parameter-tuning-for-svc-758215394769

https://stackoverflow.com/questions/56449262/how-to-upload-folders-to-google-colab

https://github.com/datarail/datarail/issues/39

https://dbader.org/blog/python-check-if-file-exists

https://datacarpentry.org/python-socialsci/11-joins/index.html

https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html

https://www.tutorialspoint.com/matplotlib/matplotlib_bar_plot.htm

https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.butter.html

https://stackoverflow.com/questions/32194967/how-to-do-pca-and-svm-for-classification-in-python

https://stackoverflow.com/questions/6822725/rolling-or-sliding-window-iterator

https://scikit-learn.org/stable/modules/cross_validation.html

https://stackoverflow.com/questions/47684606/merge-cells-with-pandas

https://scientificallysound.org/2016/08/18/python-analysing-emg-signals-part-3/

https://stackoverflow.com/questions/58374492/python-valueerror-the-length-of-the-input-vector-x-must-be-greater-than-padle

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html

https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

https://statinfer.com/204-4-2-calculating-sensitivity-and-specificity-in-python/

https://www.datacamp.com/community/tutorials/pandas-multi-index

https://www.geeksforgeeks.org/how-to-get-rows-index-names-in-pandas-dataframe/

https://stackoverflow.com/questions/28140771/select-only-one-index-of-multiindex-dataframe

https://www.educative.io/edpresso/how-to-create-a-confusion-matrix-in-python-using-scikit-learn

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

http://www.datasciencemadesimple.com/standard-deviation-function-python-pandas-row-column/

https://stackoverflow.com/questions/39047915/concat-series-onto-dataframe-with-column-name

https://www.programiz.com/python-programming/methods/list/index

https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings

https://www.phusewiki.org/wiki/index.php?title=Program_Header

https://github.com/scikit-learn-contrib/skope-rules/issues/18

https://github.com/scikit-learn/scikit-learn/issues/13176

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

https://github.com/scikit-learn/scikit-learn/issues/11020

https://stackoverflow.com/questions/1557571/how-do-i-get-time-of-a-python-programs-execution

https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Transfer project from GitHub to GitLab https://docs.gitlab.com/ee/user/project/import/github.html
